### SocialAL Models
# Data simulation and parameter recovery - multiple subjects
KLS 8.30.19; update 3.5.22  
Project info: https://osf.io/b48n2/

Model modified from :
Fareri, D. S., Chang, L. J., & Delgado, M. R. (2012). Effects of direct social experience on trust decisions and neural reward circuitry. Frontiers in Neuroscience, 6, 1–17. https://doi.org/10.3389/fnins.2012.00148

### Python version

In [1]:
import sys
print(sys.version)  

3.7.4 (default, Aug 13 2019, 15:17:50) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


### Load modules

In [2]:
import numpy as np
import random
import math
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import itertools

## Pull in functions

In [3]:
run common_functions.ipynb

In [4]:
run double_alpha_functions.ipynb

## Functions to simulate data

In [5]:
def select_response(prob):     
    n = random.uniform(0,1)
    if n > prob:
        response = 0
    else:
        response = 1
    return response

### Set constants

In [6]:
recip_rates = {0: 0.93, 1:0.6, 2:0.07}

### New function to simulate data for one sub

In [7]:
def sim_data(tn, params):
    # tn: number of trials desired
    # params: ground truth of parameters
    a_gain = params[0]
    a_loss = params[1]
    beta = params[2]
    
    # initialize variables
    Probs = [0.5, 0.5, 0.5]
    EVs = [[9,9,9,9],[9,9,9,9],[9,9,9,9]]
    
    # generate trial sequence
    trial_sequence = np.repeat([0,1,2], tn)
    random.shuffle(trial_sequence) #print(trial_sequence)
    
    choices = []
    responses = []
    
    for x in range(0,len(trial_sequence)):
        t = trial_sequence[x] 
        
        # Make a choice
        choice = action_selection(get_action_selection_probs(beta, EVs[t]))
        choices.append(choice) 
    
        # Get a response
        recip_rate = recip_rates.get(t) 
        
        response = select_response(recip_rate) 
        responses.append(response)
    
        # after choice, update probability
        if choice != 1:
            Probs[t] = update_prob(response, Probs[t], a_gain, a_loss) 
        # then update value
        EVs[t] = update_value(Probs[t]) 

    data = {'Stim_Sequence': trial_sequence, 'Choice' : choices, 'Trustee_Response': responses}    
    return data

In [8]:
#sim_data(15,[.2,.3,2])

### New function to sim data for multiple subjects

In [9]:
def sim_mult_subject_data(n, tn, params):
    data = pd.DataFrame(columns= ['Stim_Sequence', 'Choice', 'Trustee_Response', 'Subject', 'Alpha_gain', 'Alpha_loss','Beta'])
    for x in range(1, n+1):
        sdt = sim_data(tn, params)
        sdt = pd.DataFrame.from_dict(sdt)
        sdt['Subject'] = [x for t in range(0,tn*3)]
        sdt['Alpha_gain'] = params[0]
        sdt['Alpha_loss'] = params[1]
        sdt['Beta'] = params[2]
        data = pd.concat([data,sdt])
    return data

In [10]:
#data = sim_mult_subject_data(100, 15, [.2,.3, 2])
#data

### Create param space

In [11]:
# create parameter space
alpha_gains = [round(x * 0.1,1) for x in range(0,11)]
alpha_loses = [round(x * 0.1,1) for x in range(0,11)]
betas = [x * 2 for x in range(0,11)]
combos = list(itertools.product(alpha_gains, alpha_loses, betas))
#for r in combos: print(r)

### Simulate data

In [20]:
data = pd.DataFrame(columns= ['Stim_Sequence', 'Choice', 'Trustee_Response', 'Subject', 'Alpha_gain', 'Alpha_loss','Beta'])
for p in range(len(combos)):
    dt = sim_mult_subject_data(1, 15, combos[p])
    dt['Subject'] = dt['Subject'] + p
    data = pd.concat([data, dt])
data

data.to_csv(path_or_buf = '../../output/simulation/sim_2alpha_model_data.csv', index = False)

### New function to fit model to multiple subjects

In [21]:
def model_fit_mult_subject(data):
    pdt = pd.DataFrame(columns = ['Subject', 'a_gain', 'a_loss', 'beta'])
    a = pd.unique(data['Subject'])
    print('Number of Subs: ', len(a))
    for sub in range(1,len(a)+1):
        print('Subject: ', sub)
        df = data[data['Subject']==sub]
        dt = df.to_dict()
        print(dt)
        params = model_fit(dt)
        line = {'Subject': sub, 'a_gain': params[0], 'a_loss': params[1], 'beta':params[2]}
        pdt = pdt.append(line, ignore_index=True)   
    return(pdt)


In [22]:
precover = model_fit_mult_subject(data)
precover.to_csv(path_or_buf = '../../output/simulation/sim_2alpha_model_fit.csv', index = False)

Number of Subs:  1331
Subject:  1
{'Stim_Sequence': {0: 0, 1: 1, 2: 0, 3: 0, 4: 0, 5: 0, 6: 2, 7: 1, 8: 1, 9: 0, 10: 1, 11: 1, 12: 2, 13: 1, 14: 1, 15: 1, 16: 0, 17: 1, 18: 0, 19: 2, 20: 1, 21: 0, 22: 0, 23: 1, 24: 2, 25: 2, 26: 2, 27: 2, 28: 0, 29: 2, 30: 1, 31: 0, 32: 0, 33: 1, 34: 2, 35: 2, 36: 1, 37: 0, 38: 1, 39: 2, 40: 2, 41: 2, 42: 0, 43: 2, 44: 2}, 'Choice': {0: 1, 1: 2, 2: 1, 3: 4, 4: 4, 5: 1, 6: 2, 7: 2, 8: 4, 9: 1, 10: 2, 11: 2, 12: 1, 13: 1, 14: 4, 15: 1, 16: 4, 17: 3, 18: 4, 19: 3, 20: 1, 21: 2, 22: 1, 23: 2, 24: 1, 25: 2, 26: 2, 27: 4, 28: 1, 29: 3, 30: 3, 31: 4, 32: 2, 33: 3, 34: 4, 35: 3, 36: 1, 37: 1, 38: 3, 39: 3, 40: 2, 41: 4, 42: 4, 43: 3, 44: 1}, 'Trustee_Response': {0: 1, 1: 1, 2: 1, 3: 0, 4: 1, 5: 1, 6: 1, 7: 0, 8: 0, 9: 1, 10: 1, 11: 1, 12: 0, 13: 1, 14: 0, 15: 0, 16: 1, 17: 0, 18: 1, 19: 0, 20: 1, 21: 1, 22: 1, 23: 1, 24: 0, 25: 0, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 1, 33: 1, 34: 0, 35: 0, 36: 1, 37: 1, 38: 0, 39: 0, 40: 0, 41: 0, 42: 1, 43: 0, 44: 0

Subject:  5
{'Stim_Sequence': {0: 1, 1: 2, 2: 0, 3: 1, 4: 2, 5: 2, 6: 2, 7: 1, 8: 2, 9: 2, 10: 1, 11: 0, 12: 0, 13: 0, 14: 1, 15: 0, 16: 2, 17: 0, 18: 2, 19: 2, 20: 1, 21: 1, 22: 1, 23: 0, 24: 1, 25: 2, 26: 2, 27: 1, 28: 0, 29: 1, 30: 2, 31: 0, 32: 2, 33: 2, 34: 0, 35: 0, 36: 0, 37: 0, 38: 1, 39: 2, 40: 0, 41: 1, 42: 1, 43: 1, 44: 0}, 'Choice': {0: 1, 1: 3, 2: 2, 3: 1, 4: 4, 5: 2, 6: 2, 7: 1, 8: 3, 9: 2, 10: 2, 11: 3, 12: 2, 13: 2, 14: 3, 15: 3, 16: 3, 17: 1, 18: 4, 19: 4, 20: 4, 21: 3, 22: 3, 23: 3, 24: 3, 25: 4, 26: 4, 27: 4, 28: 1, 29: 4, 30: 4, 31: 2, 32: 4, 33: 3, 34: 4, 35: 2, 36: 3, 37: 2, 38: 2, 39: 4, 40: 1, 41: 3, 42: 2, 43: 3, 44: 2}, 'Trustee_Response': {0: 1, 1: 0, 2: 1, 3: 1, 4: 0, 5: 0, 6: 0, 7: 1, 8: 0, 9: 0, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 0, 17: 1, 18: 0, 19: 0, 20: 0, 21: 0, 22: 1, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 1, 32: 0, 33: 0, 34: 1, 35: 1, 36: 1, 37: 1, 38: 1, 39: 0, 40: 0, 41: 1, 42: 1, 43: 1, 44: 1}, 'Subject': {0: 5, 1

KeyboardInterrupt: 

In [ ]:
print(precover)